<a href="https://colab.research.google.com/github/OsheenArya/SpeechEmotionRecognitionCapstone/blob/main/feature_extraction_DS440.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
pip install librosa soundfile numpy sklearn pyaudio

In [44]:
import librosa
import soundfile
import os, glob, pickle
import numpy as np
from sklearn.model_selection import train_test_split
from sklearn.neural_network import MLPClassifier
from sklearn.metrics import accuracy_score

In [75]:
def extract_feature(file_name, mfcc, chroma, mel):
  X, sample_rate = librosa.load(os.path.join(file_name), res_type = "kaiser_fast")
  result = np.array([])
  if mfcc:
    mfccs=np.mean(librosa.feature.mfcc(y=X, sr=sample_rate, n_mfcc=40).T, axis=0)
    result=np.hstack((result, mfccs))
  if chroma:
    stft=np.abs(librosa.stft(X))
    chroma=np.mean(librosa.feature.chroma_stft(S=stft, sr=sample_rate).T,axis=0)
    result=np.hstack((result, chroma))
  if mel:
    mel=np.mean(librosa.feature.melspectrogram(X, sr=sample_rate).T,axis=0)
    result=np.hstack((result, mel))
  return result

In [77]:
emotions={
  '01':'neutral',
  '02':'calm',
  '03':'happy',
  '04':'sad',
  '05':'angry',
  '06':'fearful',
  '07':'disgust',
  '08':'surprised'
}

observed_emotions = ['angry', 'disgust', 'fear', 'happy', 'sad', 'neutral', 'surprise']

In [78]:
from google.colab import drive
drive.mount('/gdrive', force_remount=True)

def load_data(test_size):
    x,y=[],[]
    for file in glob.glob("/gdrive/MyDrive/RAVDESS/Actor_*/*.wav"):
        file_name=os.path.basename(file)
        emotion=emotions[file_name.split("-")[2]]
        if emotion not in observed_emotions:
            continue
        feature=extract_feature(file, mfcc=True, chroma=True, mel=True)
        x.append(feature)
        y.append(emotion)
          
    return train_test_split(np.array(x), y, test_size=test_size, random_state=9)



Mounted at /gdrive


In [79]:
x_train,x_test,y_train,y_test = load_data(test_size=0.20)


In [80]:
model=MLPClassifier(alpha=0.1, batch_size=256, epsilon=1e-08, hidden_layer_sizes=(300,), learning_rate='adaptive', max_iter=1000)

model.fit(x_train,y_train)


MLPClassifier(alpha=0.1, batch_size=256, hidden_layer_sizes=(300,),
              learning_rate='adaptive', max_iter=1000)

In [81]:
y_pred=model.predict(x_test)



In [82]:
accuracy=accuracy_score(y_true=y_test, y_pred=y_pred)


In [83]:
accuracy

0.5375722543352601